# 🧠 Etapa 1: Análise Exploratória de Dados (EDA)

## 🎯 Objetivos

Nesta etapa, o objetivo é realizar uma investigação completa no dataset para entender suas características, identificar problemas de qualidade (valores faltantes, *outliers*) e descobrir os primeiros *insights* que guiarão as próximas etapas de pré-processamento e modelagem. O foco é **analisar**, e não modificar os dados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Carregar o arquivo CSV
try:
    # Assumindo que o arquivo 'salary_prediction.csv' está no diretório correto
    df = pd.read_csv('salary_prediction.csv')
    print("Dataset carregado com sucesso!")
except FileNotFoundError:
    print("Erro: O arquivo 'salary_prediction.csv' não foi encontrado. Verifique o caminho.")
    df = None

---

## 1. Carregamento e Visão Geral dos Dados 📊

Inspeção inicial para entender a estrutura do dataset, tipos de dados e estatísticas descritivas.

In [ ]:
if df is not None:
    print("\n### Primeiras 5 linhas (.head()) ###")
    print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

    print("\n### Últimas 5 linhas (.tail()) ###")
    print(df.tail().to_markdown(index=False, numalign="left", stralign="left"))

    print("\n### Estrutura e Tipos de Dados (.info()) ###")
    df.info()

    print(f"\nNúmero de Linhas e Colunas (.shape): {df.shape}")

    print("\n### Estatísticas Descritivas (.describe()) ###")
    # Remove a coluna 'professional_id' para o describe
    desc_df = df.drop(columns=['professional_id'], errors='ignore')
    print(desc_df.describe().T.to_markdown(numalign="left", stralign="left"))

### 📝 Conclusões - Visão Geral

* **Descrição do Dataset:** O dataset contém dados de profissionais de tecnologia, abrangendo anos de experiência, nível de educação, área, senioridade, etc., e o **salário anual em BRL** como variável alvo.
* **Dimensão:** O conjunto de dados possui **2334 linhas** e **18 colunas**.
* **Variável Alvo:** **`annual_salary_brl`** (Salário Anual em BRL).
* **Observação Estatística:** O salário médio é de $\text{R}\$142.200$, com um **alto desvio padrão** (cerca de $\text{R}\$60.400$), indicando grande dispersão de salários. O valor máximo de $\text{R}\$350.000$ sugere a presença de profissionais de alta remuneração.

---

## 2. Análise de Qualidade dos Dados 🔎

Investigação de problemas comuns como valores faltantes (`NaN`) e valores atípicos (*outliers*).

In [ ]:
# 2.1. Cálculo de Valores Faltantes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_data = pd.DataFrame({
    'Total Faltante': missing_values,
    'Porcentagem (%)': missing_percentage
}).sort_values(by='Porcentagem (%)', ascending=False)

# Filtrar apenas colunas com valores faltantes
missing_data = missing_data[missing_data['Total Faltante'] > 0]

print("### Valores Faltantes (Total e Porcentagem) ###")
print(missing_data.to_markdown(numalign="left", stralign="left"))

# 2.2. Visualização de Valores Faltantes
if not missing_data.empty:
    plt.figure(figsize=(10, 6))
    sns.barplot(
        x=missing_data.index,
        y='Porcentagem (%)',
        data=missing_data,
        palette='viridis'
    )
    plt.title('Porcentagem de Valores Faltantes por Coluna')
    plt.ylabel('Porcentagem (%)')
    plt.xlabel('Colunas')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# 2.3. Boxplots para Identificação de Outliers
# Identificar colunas numéricas (excluindo o ID)
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

plt.figure(figsize=(18, 12))
for i, col in enumerate(numeric_cols):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(y=df[col], color='skyblue')
    plt.title(f'Boxplot de {col}', fontsize=12)
    plt.ylabel('')
    plt.xlabel('')

plt.suptitle('Boxplots para Identificação de Outliers em Variáveis Numéricas', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 📝 Conclusões - Qualidade dos Dados

#### Valores Faltantes
* Três colunas contêm valores nulos:
    * **`english_level`**: 11.78% dos dados estão faltando.
    * **`frameworks_known`** e **`programming_languages`**: 5.74% faltante em ambas. A coincidência sugere que **os mesmos profissionais** não preencheram essas informações.
* **Hipótese Inicial:** A ausência de dados pode ser por falha na coleta ou porque a informação não era aplicável (e.g., `'Básico'` ou `'Nenhum'` foram deixados em branco).

#### Outliers
* **`annual_salary_brl`**: Apresenta *outliers* na cauda superior (salários muito altos). Estes devem ser mantidos, pois representam a realidade do mercado.
* **`github_contributions`** e **`projects_completed`**: Demonstram uma grande quantidade de *outliers* na cauda superior, indicando a existência de profissionais com produtividade ou engajamento em projetos muito acima da média.

---

## 3. Análise Univariada 📊

Análise da distribuição individual de cada variável para entender seu comportamento.

In [ ]:
# Pré-processamento simples para lidar com whitespace em categóricas (melhor visualização)
df_clean = df.copy()
for col in df_clean.select_dtypes(include='object').columns:
    # Remove espaços em branco antes e depois dos valores categóricos
    df_clean[col] = df_clean[col].str.strip()

# 3.1. Distribuição da Variável Alvo
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['annual_salary_brl'], kde=True, bins=30, color='teal')
plt.title('Distribuição da Variável Alvo: Salário Anual (BRL)')
plt.xlabel('Salário Anual (BRL)')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# 3.2. Histogramas das Variáveis Numéricas
numeric_cols_no_target = [col for col in numeric_cols if col != 'annual_salary_brl']

plt.figure(figsize=(18, 12))
for i, col in enumerate(numeric_cols_no_target):
    plt.subplot(3, 3, i + 1)
    sns.histplot(df_clean[col].dropna(), kde=True, bins=20, color='darkorange')
    plt.title(f'Distribuição de {col}', fontsize=12)
    plt.xlabel(col)
    plt.ylabel('Frequência')

plt.suptitle('Histogramas das Variáveis Numéricas', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 3.3. Gráficos de Barras das Variáveis Categóricas
categorical_cols = [col for col in df_clean.select_dtypes(include='object').columns.tolist() if col != 'professional_id']

plt.figure(figsize=(18, 15))
for i, col in enumerate(categorical_cols):
    plt.subplot(3, 3, i + 1)
    df_clean[col].value_counts().plot(kind='bar', color=sns.color_palette("pastel")[i % 9])
    plt.title(f'Contagem de {col}', fontsize=12)
    plt.xlabel(col)
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right')

plt.suptitle('Gráficos de Barras das Variáveis Categóricas', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 📝 Conclusões - Análise Univariada

#### Variável Alvo
* **`annual_salary_brl`**: A distribuição é **assimétrica positiva (skewed right)**. Isso indica que a maioria dos profissionais se encontra nas faixas salariais mais baixas, o que é típico de dados de renda. Essa assimetria pode requerer uma **transformação logarítmica** antes da modelagem.

#### Variáveis Preditoras
* **`years_experience`**: Distribuição relativamente uniforme, com a frequência decaindo levemente para maior experiência.
* **`area`**: **Desenvolvimento** e **Dados** são as áreas mais representadas, indicando um **desbalanceamento de classes** que deve ser considerado.
* **`seniority`**: Níveis **Pleno** e **Sênior** são os mais comuns, correspondendo à maior parcela da amostra.
* **`work_mode`**: O modo **Remoto** é o mais frequente, refletindo a tendência atual do mercado de TI.

---

## 4. Análise Bivariada 🤝

Investigação da relação entre pares de variáveis para encontrar padrões e correlações.

In [ ]:
# 4.1. Matriz de Correlação (Heatmap)
correlation_matrix = df_clean[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(
    correlation_matrix,
    annot=True,
    fmt=".2f",
    cmap='coolwarm',
    cbar=True,
    linewidths=.5,
    linecolor='black'
)
plt.title('Matriz de Correlação de Pearson entre Variáveis Numéricas')
plt.show()

In [ ]:
# 4.2. Scatter Plots com Variáveis de Maior Correlação com o Salário

plt.figure(figsize=(15, 6))

# Scatter Plot 1: Salário vs. Anos de Experiência (Correlação: 0.88)
plt.subplot(1, 2, 1)
sns.scatterplot(x='years_experience', y='annual_salary_brl', data=df_clean, alpha=0.6, color='darkgreen')
plt.title('Salário vs. Anos de Experiência (Correlação: 0.88)')
plt.xlabel('Anos de Experiência')
plt.ylabel('Salário Anual (BRL)')

# Scatter Plot 2: Salário vs. Projetos Concluídos (Correlação: 0.58)
plt.subplot(1, 2, 2)
sns.scatterplot(x='projects_completed', y='annual_salary_brl', data=df_clean, alpha=0.6, color='darkred')
plt.title('Salário vs. Projetos Concluídos (Correlação: 0.58)')
plt.xlabel('Projetos Concluídos')
plt.ylabel('Salário Anual (BRL)')

plt.tight_layout()
plt.show()

In [ ]:
# 4.3. Boxplots Comparativos com Variáveis Categóricas Chave

plt.figure(figsize=(18, 6))

# Boxplot 1: Salário vs. Senioridade
plt.subplot(1, 2, 1)
# Ordenando para uma visualização lógica
order_seniority = ['Estágio', 'Júnior', 'Pleno', 'Sênior', 'Especialista', 'Diretor']
sns.boxplot(
    x='seniority',
    y='annual_salary_brl',
    data=df_clean, 
    order=order_seniority,
    palette='Set2'
)
plt.title('Salário por Nível de Senioridade')
plt.xlabel('Senioridade')
plt.ylabel('Salário Anual (BRL)')

# Boxplot 2: Salário vs. Nível de Inglês
plt.subplot(1, 2, 2)
# Ordenando para uma visualização lógica
order_english = ['Básico', 'Intermediário', 'Avançado', 'Fluente']
sns.boxplot(
    x='english_level',
    y='annual_salary_brl',
    data=df_clean, 
    order=order_english,
    palette='Set1'
)
plt.title('Salário por Nível de Inglês')
plt.xlabel('Nível de Inglês')
plt.ylabel('Salário Anual (BRL)')

plt.tight_layout()
plt.show()

### 📝 Conclusões - Análise Bivariada

#### Relações Numéricas
* **Variável mais correlacionada:** **`years_experience`** ($0.88$). Esta forte correlação positiva indica que o tempo de experiência é o principal fator numérico a prever o salário.
* **Correlações Moderadas:** **`projects_completed`** ($0.58$) e **`certifications`** ($0.45$), sugerindo que a experiência medida em projetos e certificações também impulsiona o salário.

#### Relações Categóricas
* **Impacto de `seniority`:** **Muito Forte.** A mediana e a dispersão salarial aumentam consistentemente em cada nível de senioridade, confirmando a importância dessa *feature*.
* **Impacto de `english_level`:** **Forte.** Observa-se uma clara tendência de aumento na mediana salarial à medida que o nível de inglês avança (Básico $\to$ Fluente), destacando a importância da proficiência em inglês no mercado de trabalho de TI.

---

## ✅ Resumo das Conclusões e Próximos Passos

| Seção | Conclusão Principal | Impacto no Projeto | 
| :--- | :--- | :--- |
| **Visão Geral** | Variável alvo (**`annual_salary_brl`**) tem alta variabilidade. | Necessidade de um modelo robusto para capturar a dispersão. |
| **Qualidade dos Dados** | Valores faltantes em **`english_level`**, **`frameworks_known`** e **`programming_languages`**. Outliers presentes. | As colunas com dados faltantes precisarão de **imputação**. Os outliers serão mantidos. |
| **Análise Univariada** | Salário é **assimétrico positivo**. Classes de **`area`** são desbalanceadas (predomínio de Desenvolvimento e Dados). | A assimetria do alvo pode exigir **transformação logarítmica**. |
| **Análise Bivariada** | **`years_experience`** ($0.88$) e **`seniority`** são as *features* mais importantes para prever o salário. | Priorizar essas *features* na engenharia e seleção de modelos. |

### Próximo Passo

A próxima etapa será o **Pré-processamento dos Dados** (`notebooks/02_preprocessing.ipynb`), onde aplicaremos as transformações necessárias para preparar os dados para a modelagem.